In [1]:
import os
import polars as pl
import numpy as np
import torch.nn as nn
from polars import DataFrame
import torch
from torch.utils.data import DataLoader
from typing import Tuple
import wandb

from path import RESULT_DIRECTORY
from develop import reload_function, reload_module

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
from data.source.pg_experiment import get_pg_experiment_dataframe

df_pron, _ = get_pg_experiment_dataframe(".ogg")
dataframe = df_pron.with_columns(word_id = pl.struct("word_id").rank("dense"))
dataframe = dataframe.filter(pl.col("stage") == 1)


N_WORDS = dataframe.select(pl.col("word_id").n_unique()).to_numpy()[0][0]
print(f"Number of unique words: {N_WORDS}")
print(f"Number of samples: {dataframe.shape[0]}")
print(f"Number of speakers: {dataframe.select(pl.col('id_student').n_unique()).to_numpy()[0][0]}")

Number of unique words: 12
Number of samples: 5855
Number of speakers: 547


In [3]:
def split(df: pl.DataFrame, label_col: str, train_frac=0.8, val_frac=0.1, seed=42) -> Tuple[pl.DataFrame, pl.DataFrame, pl.DataFrame]:
    classes = df.select(label_col).unique().to_series()
    train_rows, val_rows, test_rows = [], [], []
    
    rng = np.random.RandomState(seed)
    
    for cls in classes:
        class_df = df.filter(pl.col(label_col) == cls)
        n = class_df.height
        indices = rng.permutation(n)
        
        train_end = int(train_frac * n)
        val_end = int((train_frac + val_frac) * n)
        
        train_rows.append(class_df[indices[:train_end]])
        val_rows.append(class_df[indices[train_end:val_end]])
        test_rows.append(class_df[indices[val_end:]])
    
    train_df = pl.concat(train_rows)
    val_df = pl.concat(val_rows)
    test_df = pl.concat(test_rows)
    
    
    return train_df, val_df, test_df

In [4]:
from dataset import Cast, TorchDataset
from pytorch_dataloader import build_collate_fn, PaddingCollate, DefaultCollate
from transformation import Channels, RMSEnergy, TorchVadMFCC, ZeroCrossingRate

TRAIN_SPLIT = 0.6
VAL_SPLIT = 0.2
TEST_SPLIT = 1 - TRAIN_SPLIT - VAL_SPLIT
train_pl, val_pl, test_pl = split(dataframe, label_col="word_id", train_frac=TRAIN_SPLIT, val_frac=VAL_SPLIT)

def to_dataset(dataframe: DataFrame) -> TorchDataset:
    return TorchDataset(
    Cast(dataframe.get_column("rec_path"), Channels("stack","multiply")(
            TorchVadMFCC(delta=0),
        )),
    Cast(dataframe.get_column("rec_path"), Channels("cat","multiply")(
            ZeroCrossingRate(),
            RMSEnergy(),
        )),
    Cast(dataframe.get_column("word_id"), lambda x: torch.tensor(x-1, dtype=torch.long)),
    Cast(dataframe.get_column("value"), lambda x: torch.tensor(x).float()),
)

collate_fn = build_collate_fn(
    PaddingCollate(mode="SET_MAX_LEN", max_len=80, pad_dim=2),
    PaddingCollate(mode="SET_MAX_LEN", max_len=80, pad_dim=1),
    DefaultCollate(),
    DefaultCollate(),
)
dataset_train = to_dataset(train_pl)
dataset_val = to_dataset(val_pl)
dataset_test = to_dataset(test_pl)

In [ ]:
from os import name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#note, if you are using Windows you MUST set `num_workers=0` - TL;DT multithreading DON'T work in notebooks because Windows DON'T have `fork()`
num_workers = 0 if name == "nt" else 4
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=collate_fn, num_workers=num_workers)
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)

for x in next(iter(train_loader)):
    print(x.shape)

In [ ]:
from pytorch_dataloader import MemoryLoadedDataLoader
train_loader = MemoryLoadedDataLoader(train_loader, device=device)
print("Loaded train loader into memory")
val_loader = MemoryLoadedDataLoader(val_loader, device=device)
print("Loaded validation loader into memory")
test_loader = MemoryLoadedDataLoader(test_loader, device=device)

In [7]:
from models.FusionCNN import ContextFusionCNN, ContextCNN
from models.Guesser import ContextGuesser

cfcnn_model = ContextFusionCNN(n_2d_channels=1, n_1d_channels=2, num_words=N_WORDS, dropout_rate=0.5)
ccnn_model = ContextCNN(n_2d_channels=1, num_words=N_WORDS, dropout_rate=0.5)
guesser_model = ContextGuesser(num_words=N_WORDS)
models = [guesser_model, cfcnn_model, ccnn_model]

## Hyperparameters

In [8]:
from type_helpers import Kwargs

epochs = 140

Optimizer = torch.optim.Adam
OptimizerKwargs = Kwargs(Optimizer)
optimizer_config = OptimizerKwargs(
    lr=1e-4,
    weight_decay= 1e-4,
)

Scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau
SchedulerKwargs = Kwargs(Scheduler)

scheduler_config = SchedulerKwargs(
    mode='min',
    factor=0.5,
    patience=5
)
criterion = nn.BCELoss()


In [9]:
from models.SimpleCNN_v2 import train, evaluate

def train_model(model):
    model = model.to(device)
    optimizer = Optimizer(model.parameters(), **optimizer_config)
    scheduler = Scheduler(optimizer, **scheduler_config)

    run = wandb.init(
        # name of the run
        name=f"14+11 (0.5 dropout) experiment - {model.__class__.__name__}",
        config={
            "Name": f"{model.__class__.__name__}",
            "learning_rate": optimizer_config["lr"],
            "optimizer": f"{Optimizer.__name__}",
            "criterion": f"{criterion.__class__.__name__}",
            "architecture": f"{model.__class__.__name__}",
            "architecture_details": str(model),
            "dataset": "Stage-I",
            "train_val_test(%)": f'{TRAIN_SPLIT}-{VAL_SPLIT}-{TEST_SPLIT}',
            "epochs": epochs,
        },
    )

    # Training loop
    for epoch in range(epochs):
        for label_version in ["v2"]:
            train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
            val_loss, val_acc = evaluate(model, val_loader, criterion, device)
            scheduler.step(val_loss)
        run.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, })
        print(
            f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    run.log({"model_eval": model.eval()})
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    run.log({"test_acc": test_acc, "test_loss": test_loss})
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    # Saving the model to pth and adding it to the artifacts of the run, there is 5GB of memory on wandb, so we should be fine.
    torch.save(model.state_dict(), os.path.join(RESULT_DIRECTORY, f"{model.__class__.__name__}.pth"))
    artifact = wandb.Artifact(f"{model.__class__.__name__}-model", type="model")
    artifact.add_file(os.path.join(RESULT_DIRECTORY, f"{model.__class__.__name__}.pth"))
    run.log_artifact(artifact)

    # Finish the run so it gets sent to the remote. You can discover the run right after that on the dashboard.
    run.finish()
    return test_acc


In [10]:
test_accuracies = {}
for model in models:
    test_acc = train_model(model)
    test_accuracies[model.__class__.__name__] = test_acc 

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Epoch 1, Train Loss: 0.6883, Train Acc: 0.5305, Val Loss: 0.6793, Val Acc: 0.6097
Epoch 2, Train Loss: 0.6798, Train Acc: 0.5858, Val Loss: 0.6731, Val Acc: 0.6072
Epoch 3, Train Loss: 0.6740, Train Acc: 0.5838, Val Loss: 0.6677, Val Acc: 0.5816
Epoch 4, Train Loss: 0.6686, Train Acc: 0.5898, Val Loss: 0.6626, Val Acc: 0.5816
Epoch 5, Train Loss: 0.6657, Train Acc: 0.5912, Val Loss: 0.6579, Val Acc: 0.5816
Epoch 6, Train Loss: 0.6577, Train Acc: 0.5949, Val Loss: 0.6531, Val Acc: 0.5952
Epoch 7, Train Loss: 0.6547, Train Acc: 0.6001, Val Loss: 0.6487, Val Acc: 0.5952
Epoch 8, Train Loss: 0.6501, Train Acc: 0.6075, Val Loss: 0.6446, Val Acc: 0.6003
Epoch 9, Train Loss: 0.6481, Train Acc: 0.6023, Val Loss: 0.6410, Val Acc: 0.6260
Epoch 10, Train Loss: 0.6402, Train Acc: 0.6129, Val Loss: 0.6375, Val Acc: 0.6285
Epoch 11, Train Loss: 0.6386, Train Acc: 0.6140, Val Loss: 0.6347, Val Acc: 0.6285
Epoch 12, Train Loss: 0.6392, Train Acc: 0.6129, Val Loss: 0.6325, Val Acc: 0.6285
Epoch 13, Tra

test_acc,▁
test_loss,▁
train_acc,▁▅▆▆▇▇██▇▇▇█▇▇▇██▇▇█▇▇▇▇▇▇█▇█▇▇▇▇▇▇▇██▇▇
train_loss,█▇▆▅▃▃▂▂▂▂▁▁▂▂▁▁▁▁▂▁▁▁▁▂▂▁▁▂▁▂▂▁▁▁▁▂▂▁▁▁
val_acc,▄▄▁▃▆███████████████████████████████████
val_loss,█▆▅▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model_eval,ContextGuesser( (w...
test_acc,0.64371
test_loss,0.62717
train_acc,0.63056
train_loss,0.62274


wandb: Currently logged in as: tadziobrzeski (fischbach-kamil-pg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1, Train Loss: 0.6821, Train Acc: 0.5852, Val Loss: 0.6814, Val Acc: 0.5816
Epoch 2, Train Loss: 0.6789, Train Acc: 0.5852, Val Loss: 0.6776, Val Acc: 0.5816
Epoch 3, Train Loss: 0.6759, Train Acc: 0.5855, Val Loss: 0.6736, Val Acc: 0.5816
Epoch 4, Train Loss: 0.6721, Train Acc: 0.5855, Val Loss: 0.6686, Val Acc: 0.5816
Epoch 5, Train Loss: 0.6693, Train Acc: 0.5855, Val Loss: 0.6647, Val Acc: 0.5816
Epoch 6, Train Loss: 0.6657, Train Acc: 0.5867, Val Loss: 0.6608, Val Acc: 0.5816
Epoch 7, Train Loss: 0.6613, Train Acc: 0.5901, Val Loss: 0.6548, Val Acc: 0.5816
Epoch 8, Train Loss: 0.6559, Train Acc: 0.5938, Val Loss: 0.6467, Val Acc: 0.6003
Epoch 9, Train Loss: 0.6529, Train Acc: 0.5972, Val Loss: 0.6420, Val Acc: 0.6012
Epoch 10, Train Loss: 0.6504, Train Acc: 0.5983, Val Loss: 0.6368, Val Acc: 0.6038
Epoch 11, Train Loss: 0.6431, Train Acc: 0.6120, Val Loss: 0.6289, Val Acc: 0.6072
Epoch 12, Train Loss: 0.6381, Train Acc: 0.6100, Val Loss: 0.6236, Val Acc: 0.6123
Epoch 13, Tra

test_acc,▁
test_loss,▁
train_acc,▁▁▁▂▂▃▄▅▅▅▅▅▅▅▅▆▇▆▆▆▇▇▇▇▇▇▇▇▇█▇██▇██████
train_loss,██▇▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▂▂▁▁▁▂▁▁▁▁▂▂
val_acc,▁▂▂▅▅▆▅▆▆▆▇▇▇▇▇▇▇▇█▇▇████▇██████████████
val_loss,█▇▇▇▆▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model_eval,ContextFusionCNN( ...
test_acc,0.70068
test_loss,0.57273
train_acc,0.74344
train_loss,0.52815


Epoch 1, Train Loss: 0.6943, Train Acc: 0.4952, Val Loss: 0.6862, Val Acc: 0.6080
Epoch 2, Train Loss: 0.6839, Train Acc: 0.5519, Val Loss: 0.6748, Val Acc: 0.6003
Epoch 3, Train Loss: 0.6752, Train Acc: 0.5758, Val Loss: 0.6651, Val Acc: 0.6020
Epoch 4, Train Loss: 0.6741, Train Acc: 0.5775, Val Loss: 0.6611, Val Acc: 0.6089
Epoch 5, Train Loss: 0.6672, Train Acc: 0.5892, Val Loss: 0.6543, Val Acc: 0.6080
Epoch 6, Train Loss: 0.6635, Train Acc: 0.5935, Val Loss: 0.6489, Val Acc: 0.6140
Epoch 7, Train Loss: 0.6609, Train Acc: 0.6023, Val Loss: 0.6427, Val Acc: 0.6200
Epoch 8, Train Loss: 0.6513, Train Acc: 0.6060, Val Loss: 0.6387, Val Acc: 0.6362
Epoch 9, Train Loss: 0.6482, Train Acc: 0.6132, Val Loss: 0.6326, Val Acc: 0.6413
Epoch 10, Train Loss: 0.6435, Train Acc: 0.6197, Val Loss: 0.6284, Val Acc: 0.6405
Epoch 11, Train Loss: 0.6411, Train Acc: 0.6360, Val Loss: 0.6253, Val Acc: 0.6447
Epoch 12, Train Loss: 0.6380, Train Acc: 0.6254, Val Loss: 0.6197, Val Acc: 0.6499
Epoch 13, Tra

test_acc,▁
test_loss,▁
train_acc,▁▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▅▆▆▆▆▆▇▆▇▇▇▇▇▇█▇██▇██
train_loss,██▇▇▆▆▅▅▅▅▅▄▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▂▁
val_acc,▁▁▄▄▄▅▅▅▅▆▆▆▆▆▇▇▆▆▇▇▇▇▇█▇█▇█▇██▇▇███████
val_loss,█▇▇▆▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model_eval,ContextCNN( (featu...
test_acc,0.71684
test_loss,0.56419
train_acc,0.75656
train_loss,0.50345


In [11]:
for model_name, acc in test_accuracies.items():
    print(f"Model: {model_name}, Test Accuracy: {acc:.4f}")

Model: ContextGuesser, Test Accuracy: 0.6437
Model: ContextFusionCNN, Test Accuracy: 0.7007
Model: ContextCNN, Test Accuracy: 0.7168


In [12]:
for model in models:
    model = model.to(device)
    confusion_matrix = {
        "True_Positive": 0,
        "True_Negative": 0,
        "False_Positive": 0,
        "False_Negative": 0,
    }

    with torch.no_grad():
        for *specs, labels in test_loader:
            specs = (spec.to(device) for spec in specs)
            labels = labels.to(device)
            outputs = model(*specs)
            preds = (outputs >= 0.5).float()
            confusion_matrix["True_Positive"] += ((preds == 1) & (labels == 1)).sum().item()
            confusion_matrix["True_Negative"] += ((preds == 0) & (labels == 0)).sum().item()
            confusion_matrix["False_Positive"] += ((preds == 1) & (labels == 0)).sum().item()
            confusion_matrix["False_Negative"] += ((preds == 0) & (labels == 1)).sum().item()
    print(f"\nConfusion Matrix for {model.__class__.__name__}:")
    print(f"  True Positive:  {confusion_matrix['True_Positive']:>5}")
    print(f"  True Negative:  {confusion_matrix['True_Negative']:>5}")
    print(f"  False Positive: {confusion_matrix['False_Positive']:>5}")
    print(f"  False Negative: {confusion_matrix['False_Negative']:>5}")
    
    total = sum(confusion_matrix.values())
    accuracy = (confusion_matrix['True_Positive'] + confusion_matrix['True_Negative']) / total
    precision = confusion_matrix['True_Positive'] / (confusion_matrix['True_Positive'] + confusion_matrix['False_Positive']) if (confusion_matrix['True_Positive'] + confusion_matrix['False_Positive']) > 0 else 0
    recall = confusion_matrix['True_Positive'] / (confusion_matrix['True_Positive'] + confusion_matrix['False_Negative']) if (confusion_matrix['True_Positive'] + confusion_matrix['False_Negative']) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"\n  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    print("-" * 50)



Confusion Matrix for ContextGuesser:
  True Positive:    480
  True Negative:    277
  False Positive:   214
  False Negative:   205

  Accuracy:  0.6437
  Precision: 0.6916
  Recall:    0.7007
  F1-Score:  0.6962
--------------------------------------------------

Confusion Matrix for ContextFusionCNN:
  True Positive:    559
  True Negative:    265
  False Positive:   226
  False Negative:   126

  Accuracy:  0.7007
  Precision: 0.7121
  Recall:    0.8161
  F1-Score:  0.7605
--------------------------------------------------

Confusion Matrix for ContextCNN:
  True Positive:    536
  True Negative:    307
  False Positive:   184
  False Negative:   149

  Accuracy:  0.7168
  Precision: 0.7444
  Recall:    0.7825
  F1-Score:  0.7630
--------------------------------------------------
